# Working with data

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="..\\..\\data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets
test_data = datasets.FashionMNIST(
    root="..\\..\\data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:21<00:00, 1.25MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 143kB/s]
100%|██████████| 4.42M/4.42M [00:04<00:00, 930kB/s] 
100%|██████████| 5.15k/5.15k [00:00<?, ?B/s]


In [3]:
batch_size = 64

# Create data loaders.
train_loader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# Creating Models

- Để tạo model, định nghĩa một lớp kế thừa từ module nn.Module
- Sau đó định nghĩa các layers của model bằng hàm khởi tạo __init__
- Và định nghĩ một hàm forward để định nghĩa cách mà data đi qua model.
- Để tăng tốc quá trình huấn luyện, sử dụng CUDA, MPS, MTIA hoặc XPU. Nếu các thứ đó không có, mặc định là sử dụng CPU.

In [9]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define the model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
        
model = NeuralNetwork().to(device=device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# Optimizing the Model Parameters

- Để train một model, chúng ta cần hàm loss (loss function) và một trình tối ưu (opimizer) 

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

Trong một vòng đào tạo duy nhất, mô hình đưa ra dự đoán về tập dữ liệu đào tạo (được đưa vào theo từng đợt) và truyền ngược lỗi dự đoán để điều chỉnh các tham số của mô hình.

In [11]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"Loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            

Chúng ta cũng có thể kiểm tra hiệu suất của mô hình so vói tập dữ liệu thử nghiệm để đảm bảo mô hình đang học

In [15]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


Quá trình đào tạo được tiến hành qua nhiều lần lặp lại (epoch). Trong mỗi epoch, mô hình học các tham số để đưa ra dự đoán tốt hơn. Chúng tôi in độ chính xác và mất mát của mô hình tại mỗi epoch; chúng tôi muốn thấy độ chính xác tăng lên và mất mát giảm đi sau mỗi epoch.

In [16]:
epochs = 5
for t in range(epochs):
    train(train_loader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Loss: 2.166575 [   64/60000]
Loss: 2.163471 [ 6464/60000]
Loss: 2.107331 [12864/60000]
Loss: 2.130344 [19264/60000]
Loss: 2.079512 [25664/60000]
Loss: 2.009227 [32064/60000]
Loss: 2.061512 [38464/60000]
Loss: 1.969751 [44864/60000]
Loss: 1.977387 [51264/60000]
Loss: 1.916028 [57664/60000]
Test error: 
 Accuracy: 53.0%, Avg loss: 1.906045 

Loss: 1.931304 [   64/60000]
Loss: 1.906060 [ 6464/60000]
Loss: 1.794049 [12864/60000]
Loss: 1.843705 [19264/60000]
Loss: 1.727313 [25664/60000]
Loss: 1.673452 [32064/60000]
Loss: 1.720644 [38464/60000]
Loss: 1.602671 [44864/60000]
Loss: 1.630123 [51264/60000]
Loss: 1.532759 [57664/60000]
Test error: 
 Accuracy: 60.3%, Avg loss: 1.538773 

Loss: 1.600547 [   64/60000]
Loss: 1.566487 [ 6464/60000]
Loss: 1.419550 [12864/60000]
Loss: 1.494487 [19264/60000]
Loss: 1.368685 [25664/60000]
Loss: 1.359783 [32064/60000]
Loss: 1.392320 [38464/60000]
Loss: 1.299006 [44864/60000]
Loss: 1.334747 [51264/60000]
Loss: 1.236593 [57664/60000]
Test error: 
 Accuracy: 63

# Saving Models

In [21]:
torch.save(model.state_dict(), "..\\..\\models\\01_00_model.pth")
print("Saved PyTorch Model State to ..\\..\\models\\01_00_model.pth")

Saved PyTorch Model State to ..\..\models\01_00_model.pth


# Loading Models

In [22]:
model = NeuralNetwork()
model.load_state_dict(torch.load("..\\..\\models\\01_00_model.pth", weights_only=True))

<All keys matched successfully>

In [23]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted: '{predicted}', Actual: '{actual}'")

Predicted: 'Ankle boot', Actual: 'Ankle boot'
